In [ ]:
from flask import Flask, render_template
import sqlite3
import pandas as pd

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/data')
def data():
    conn = sqlite3.connect('nasa_data.db')
    neows_data = pd.read_sql('SELECT * FROM neows_data', conn).to_html(classes='table', index=False)
    meteorite_data = pd.read_sql('SELECT * FROM meteorite_landings', conn).to_html(classes='table', index=False)
    conn.close()
    return render_template('data.html', neows_data=neows_data, meteorite_data=meteorite_data)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)
